In [32]:
#Import Dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
import pprint
import json

#API Base URL
base_url = 'http://hotline.whalemuseum.org/api.json'
data = requests.get(base_url)
print(data)

# Pretty Print the output of the JSON
responses = requests.get(base_url).json()
print(json.dumps(responses, indent=4, sort_keys=True))

<Response [200]>
[
    {
        "created_at": "2017-11-09T18:16:30Z",
        "description": "Spotted from the Edmonds-Kingston ferry, near fishing boats.",
        "id": "5a049b7e686f743ec5040000",
        "latitude": 47.79953,
        "location": "",
        "longitude": -122.47996699999999,
        "orca_type": "unknown",
        "quantity": "5",
        "sighted_at": "2017-11-09T18:11:00Z",
        "species": "orca",
        "updated_at": "2017-11-14T22:20:45Z",
        "url": "http://hotline.whalemuseum.org/sightings/5a049b7e686f743ec5040000"
    },
    {
        "created_at": "2017-10-01T00:41:04Z",
        "description": "From the Inn at Langley looking east there was a very large group of porpoise swimming south in the Saratoga passage",
        "id": "59d039a0686f743ec5020000",
        "latitude": 48.047447813103005,
        "location": "Camano Island, WA, US",
        "longitude": -122.40477597314452,
        "orca_type": "unknown",
        "quantity": "20",
        "sighted

In [34]:
#Empty Lists to store data
lat = []
location = []
long = []
orca_type = []
quantity = []
species = []
count = 1

# Initialize loop
for response in responses:
    lat.append(response['latitude'])
    long.append(response['longitude'])
    location.append(response['location'])
    species.append(response['species'])
    quantity.append(response['quantity'])
    orca_type.append(response['orca_type'])
    count = count + 1

orca_dict = {
    'Species': species,
    'Orca Type': orca_type,
    'Quantity': quantity,
    'Location': location,
    'Latitude': lat,
    'Longitude': long

}
# Convert dictionary to dataframe
orca_data = pd.DataFrame.from_dict(orca_dict)
orca_data

,Species,Orca Type,Quantity,Location,Latitude,Longitude
0,orca,unknown,5,,47.799530,-122.479967
1,harbor porpoise,unknown,20,"Camano Island, WA, US",48.047448,-122.404776
2,humpback,unknown,1,"Gig Harbor, WA, US",47.307289,-122.561674
3,humpback,unknown,1,"Steilacoom, WA, US",47.166960,-122.614546
4,humpback,unknown,1,"Edmonds, WA, US",47.884579,-122.342120
5,orca,unknown,14,"Friday Harbor, WA, US",48.458409,-123.041211
6,gray whale,unknown,1,"Coupeville, WA, US",48.240795,-122.942763
7,orca,offshore,None,,49.650783,-124.862267
8,orca,southern resident,None,"Kingston, WA, US",47.780000,-122.440000
9,orca,southern resident,None,"Bainbridge Island, WA, US",47.620000,-122.450000


In [56]:
#Reading CSV for whale data
filename = "resources\seamap_csv.csv"
seamap_data = pd.read_csv(filename, header=0, index_col=0)
seamap_data
#seamap_data.drop(columns=['geom', 'oid', 'geom_wkt', 'csquare', 'datasets'])----- for some reason not removing columns
#seamap_data


,oid,longitude,latitude,providers,species,datasets,date_min,date_max,num_records,num_animals,num_species,csquare,geom_wkt
geom,,,,,,,,,,,,,
0103000020E6100000010000000500000000000000006066C00000000000C051C000000000006066C000000000000052C000000000008066C000000000000052C000000000008066C00000000000C051C000000000006066C00000000000C051C0,20568991,-179,-72,"David Bowden, NIWA;Rochelle Constantine, Unive...",Balaenoptera bonaerensis,103152542,2/26/2018,2/26/2018,2,4.0,1,5717:219,"POLYGON((-179 -71,-179 -72,-180 -72,-180 -71,-..."
0103000020E6100000010000000500000000000000006066C000000000008051C000000000006066C00000000000C051C000000000008066C00000000000C051C000000000008066C000000000008051C000000000006066C000000000008051C0,20568988,-179,-71,"David Bowden, NIWA;Rochelle Constantine, Unive...",Megaptera novaeangliae,103152542,2/28/2018,3/1/2018,3,6.0,1,5717:209,"POLYGON((-179 -70,-179 -71,-180 -71,-180 -70,-..."
0103000020E6100000010000000500000000000000006066C000000000004051C000000000006066C000000000008051C000000000008066C000000000008051C000000000008066C000000000004051C000000000006066C000000000004051C0,20568986,-179,-69,"David Bowden, NIWA;Rochelle Constantine, Unive...",Megaptera novaeangliae,103152542,3/2/2018,3/2/2018,1,2.0,1,5617:499,"POLYGON((-179 -69,-179 -70,-180 -70,-180 -69,-..."
0103000020E6100000010000000500000000000000006066C000000000000051C000000000006066C000000000004051C000000000008066C000000000004051C000000000008066C000000000000051C000000000006066C000000000000051C0,20568972,-179,-68,"David Bowden, NIWA;Rochelle Constantine, Unive...",Balaenoptera acutorostrata;Megaptera novaeangliae,103152363;103152542,12/23/1910,3/6/2018,6,24.0,2,5617:489,"POLYGON((-179 -68,-179 -69,-180 -69,-180 -68,-..."
0103000020E6100000010000000500000000000000006066C00000000000C050C000000000006066C000000000000051C000000000008066C000000000000051C000000000008066C00000000000C050C000000000006066C00000000000C050C0,20568968,-179,-67,"David Bowden, NIWA;Rochelle Constantine, Unive...",Megaptera novaeangliae,103152542,3/6/2018,3/6/2018,5,11.0,1,5617:479,"POLYGON((-179 -67,-179 -68,-180 -68,-180 -67,-..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
0103000020E6100000010000000500000000000000006066400000000000004F4000000000006066400000000000804F4000000000008066400000000000804F4000000000008066400000000000004F4000000000006066400000000000004F40,5814427,180,62,National Marine Mammal Laboratory;Wildlife Con...,Balaena mysticetus;Balaenoptera acutorostrata;...,821;885,6/1/1913,7/31/2010,10,21.0,4,1617:229,"POLYGON((179 62,179 63,180 63,180 62,179 62))"
0103000020E6100000010000000500000000000000006066400000000000804F40000000000060664000000000000050400000000000806640000000000000504000000000008066400000000000804F4000000000006066400000000000804F40,22746558,180,64,Russian Cetacean Habitat Project via Happywhal...,Balaena mysticetus;Megaptera novaeangliae,885;1765,6/1/1913,8/3/2005,18,18.0,2,1617:239,"POLYGON((179 63,179 64,180 64,180 63,179 63))"
0103000020E610000001000000050000000000000000606640000000000000504000000000006066400000000000405040000000000080664000000000004050400000000000806640000000000000504000000000006066400000000000005040,5814433,180,65,Wildlife Conservation Society,Balaena mysticetus,885,6/1/1913,7/1/1913,11,11.0,1,1617:249,"POLYGON((179 64,179 65,180 65,180 64,179 64))"
